In [ ]:
import pyrogue.protocols
import pyrogue.utilities.fileio
import rogue

top_level=f'{os.getcwd()}/../'
rootTopLevel = top_level+'script/'
pr.addLibraryPath( rootTopLevel )

import setupLibPaths
from dataDebug import dataDebug

In [ ]:
# Useful short names
APP  = root.App
AXIV = root.Core.AxiVersion
ASICTOP = APP.AsicTop
TRIG = ASICTOP.TriggerRegisters
ASIC0 = APP.Mv2Asic[0]
ASIC1 = APP.Mv2Asic[1]
ASIC2 = APP.Mv2Asic[2]
ASIC3 = APP.Mv2Asic[3]
HSDAC = APP.Dac.FastDac 
PKREG = [None] * 4
PKREG[0] = ASICTOP.DigAsicStrmRegisters0
PKREG[1] = ASICTOP.DigAsicStrmRegisters1
PKREG[2] = ASICTOP.DigAsicStrmRegisters2
PKREG[3] = ASICTOP.DigAsicStrmRegisters3
BATCHER0 = ASICTOP.BatcherEventBuilder0
BATCHER1 = ASICTOP.BatcherEventBuilder1
BATCHER2 = ASICTOP.BatcherEventBuilder2
BATCHER3 = ASICTOP.BatcherEventBuilder3
DEBUG0 = root._dbg[0]
DEBUG1 = root._dbg[1]
DEBUG2 = root._dbg[2]
DEBUG3 = root._dbg[3]
DATARCV0 = root.DataReceiver0
DATARCV1 = root.DataReceiver1
DATARCV2 = root.DataReceiver2
DATARCV3 = root.DataReceiver3
FULLRATERCV0 = root.fullRateDataReceiver[0]
FULLRATERCV1 = root.fullRateDataReceiver[1]
FULLRATERCV2 = root.fullRateDataReceiver[2]
FULLRATERCV3 = root.fullRateDataReceiver[3]
DAC = APP.Dac
REGCTRL = ASICTOP.RegisterControlDualClock

In [ ]:
# Set & Read ScratchPad
def probeScratchPad() :
    print(hex(AXIV.ScratchPad.get()))
    AXIV.ScratchPad.set(0xdeadbeef)
    print(hex(AXIV.ScratchPad.get()))

In [ ]:
def disableAllReceivers() :
    root.disableAndCleanAllFullRateDataRcv()
    root.enableDataRcv(False)
    root.enableDataDebug(False)

In [ ]:
def printLaneMask() :
    #Disabled lanes in ASICS
    for asicIndex in range(4) : 
        root.App.AsicTop.DigAsicStrmRegisters0.enable.set(True)
        print("ASIC{}: {:#x}".format(asicIndex, getattr(root.App.AsicTop, f"DigAsicStrmRegisters{asicIndex}").DisableLane.get()))


In [ ]:
#plot image
def plotReceiverFrames() : 
    plt.subplots(2,2,figsize=(17,17))
    for asicIndex in range(root.numOfAsics):
        if asicIndex == 3 :
            plt.subplot(2,2,3)
        elif asicIndex == 2 :
            plt.subplot(2,2,4)
        else :
            plt.subplot(2,2,asicIndex+1)
        
        if np.shape(frame[asicIndex])[0] != 1 :
            plt.imshow(frame[asicIndex])
            plt.xlabel("ASIC {}".format(asicIndex))
            plt.colorbar()
        else :
            plt.xlabel("ASIC {}: No data".format(asicIndex))

In [ ]:
def setSR0(width, delay) :
    REGCTRL.SR0Delay1.set(delay)
    REGCTRL.SR0Width1.set(width)
    print(f'SR0 Width set to {REGCTRL.SR0Width1_us.get():.3f}uS')
    print(f'SR0 Delay set to {REGCTRL.SR0Delay_us.get():.3f}uS')
    
def setAcq1(width, delay) :
    REGCTRL.AcqDelay1.set(delay)
    REGCTRL.AcqWidth1.set(width)
    print(f'Acq1 Width set to {REGCTRL.AcqWidth1_us.get():.3f}uS')
    print(f'Acq1 Delay set to {REGCTRL.AcqDelay1_us.get():.3f}uS')
    
def setAcq2(width, delay) :
    REGCTRL.AcqDelay2.set(delay)
    REGCTRL.AcqWidth2.set(width)
    print(f'Acq2 Width set to {REGCTRL.AcqWidth2_us.get():.3f}uS')
    print(f'Acq2 Delay set to {REGCTRL.AcqDelay2_us.get():.3f}uS')    

def setR0(width, delay) :
    REGCTRL.R0Delay.set(delay)
    REGCTRL.R0Width.set(width)
    print(f'R0 Width set to {REGCTRL.R0Width_us.get():.3f}uS')
    print(f'R0 Delay set to {REGCTRL.R0Delay_us.get():.3f}uS') 

    
def setSync(width, delay) :
    REGCTRL.SyncDelay.set(delay)
    REGCTRL.SyncWidth.set(width)
    print(f'Sync Delay set to {REGCTRL.SyncDelay_us.get():.3f}uS')     

In [1]:
def calculateWaveforms(intTime = 560, startTime = 10, R0AfterAcq = 1) :  # 560 = 3.6uS
    # 31250 is 200 us
    period156 = 1/156.25 # uS
    period168 = 1/168
    
    setR0(R0AfterAcq + intTime*2, startTime)
    setAcq1(intTime, intTime+startTime)
    
    print(f'-------------------------')
    print(f'Acq width = {REGCTRL.AcqWidth1.get()} ticks')
    print(f'Acq delay = {REGCTRL.AcqDelay1.get()} ticks')
    print(f'R0  width = {REGCTRL.R0Width.get()} ticks')
    print(f'R0  delay = {REGCTRL.R0Delay.get()} ticks')
    print(f'-------------------------')
    AcqEnd = REGCTRL.AcqWidth1.get() + REGCTRL.AcqDelay1.get()
    R0End = REGCTRL.R0Width.get() + REGCTRL.R0Delay.get()
    print(f'Acq width + delay = {AcqEnd}')
    print(f'R0 width + delay = {R0End}')
    print(f'-------------------------')
    SROStartDelay = int((R0End-AcqEnd)*period156/period168)
    print(f'SR0 acceptable delay > {SROStartDelay}')
    print(f'SR0 acceptable width > {int(188/period168) + 1} and less than {int(200/period168 - (R0End*period156/period168))} for SR0 delay of {int(SROStartDelay)}')
    # Print human-readable values
    print(f'-------------------------')
    baseline_int_time = REGCTRL.AcqDelay1_us.get() - REGCTRL.R0Delay_us.get()
    integration_time = REGCTRL.AcqWidth1_us.get()
    print(f'Baseline time:    {baseline_int_time} uS')
    print(f'Integration time: {integration_time} uS')
    print(f'Sampling done at: {REGCTRL.AcqWidth1_us.get()+REGCTRL.AcqDelay1_us.get()} uS')

In [ ]:
def clearUpStreamPpg():
    for i in range(4):
        root.pciePgp.Lane[i].Ctrl.CountReset()

def clearDownStreamPpg():
    for i in range(4):
        root.Core.PgpMon[i].Ctrl.CountReset()

def getUpStreamPpgFrmCnt():
    for i in range(4):
        print("Upstream pgp got {} frames".format(root.pciePgp.Lane[i].RxStatus.FrameCnt.get()))

def getDownStreamPpgFrmCnt():
    for i in range(4):
        print("Downstream pgp got {} frames".format(root.Core.PgpMon[i].TxStatus.FrameCnt.get()))
        
def clearDigAsicStrmReg():
    for i in range(4):
        PKREG[i].CountReset()

def clearSspMonGrp() :
    for i in range(4):
        root.App.SspMonGrp[i].CntRst()

def disablePpgFlowCtrl(disable):
    for i in range(4):
        root.pciePgp.Lane[i].Ctrl.FlowControlDisable.set(disable)

def getPKREGCounters(enableAsics) :
    TimeoutCntLane = [0] * 24
    LockedCnt      = [0] * 24
    BitSlipCnt     = [0] * 24
    ErrorDetCnt    = [0] * 24
    DataOvfLane    = [0] * 24    
    threshold = 1
    for asicIndex, asicEnable in enumerate(enableAsics):
        if(asicEnable == 1):
            disable = getattr(root.App.AsicTop, f"DigAsicStrmRegisters{asicIndex}").DisableLane.get()
            print("DigAsicStrmRegister{} FrameCount={} disable={}".format(asicIndex, PKREG[asicIndex].FrameCount.get(), hex(disable)))
            for i in range(24):
                if ((0x1 << i) & disable) != 0 :
                    continue
                TimeoutCntLane[i] = getattr(root.App.AsicTop, f"DigAsicStrmRegisters{asicIndex}").TimeoutCntLane[i].get()
                if(TimeoutCntLane[i]> threshold) :
                    print("ASIC {} Lane {} is having {} timeouts".format(asicIndex, i, TimeoutCntLane[i]))
    
                DataOvfLane[i] = getattr(root.App.AsicTop, f"DigAsicStrmRegisters{asicIndex}").DataOvfLane[i].get()
                if(DataOvfLane[i]> 0) :
                    print("ASIC {} Lane {} is having overflow of {}".format(asicIndex, i, DataOvfLane[i]))
                '''
                LockedCnt[i] = getattr(root.App, f"SspMonGrp[{asicIndex}]").LockedCnt[i].get()
                if(LockedCnt[i]> threshold) :
                    print("ASIC {} Lane {} is having {} locked Counts".format(asicIndex, i, LockedCnt[i]))
    
                BitSlipCnt[i] = getattr(root.App, f"SspMonGrp[{asicIndex}]").BitSlipCnt[i].get()
                if(BitSlipCnt[i]> threshold) :
                    print("ASIC {} Lane {} is having {} bitslip Counts".format(asicIndex, i, BitSlipCnt[i]))
    
                ErrorDetCnt[i] = getattr(root.App, f"SspMonGrp[{asicIndex}]").ErrorDetCnt[i].get()
                if(ErrorDetCnt[i]> threshold) :
                    print("ASIC {} Lane {} is having {} Error Counts".format(asicIndex, i, ErrorDetCnt[i]))    
                '''

In [ ]:
def clearRegisters() :
    clearUpStreamPpg()
    clearDownStreamPpg()
    clearSspMonGrp()
    clearDigAsicStrmReg()

In [ ]:
def printRegisters(asicMask = [1,1,1,1]) :
    getPKREGCounters(asicMask)
    getUpStreamPpgFrmCnt()
    getDownStreamPpgFrmCnt()

In [ ]:
numOfAsics = 4
readerReceiver = [dataDebug(name = "readerReceiver[{}]".format(lane), size = 70000) for lane in range(numOfAsics)]
filter =  [rogue.interfaces.stream.Filter(False, lane) for lane in range(numOfAsics)]
# not sure what to do this yet
cfgFilter =  rogue.interfaces.stream.Filter(False, 5)
fread = rogue.utilities.fileio.StreamReader()
readUnbatcher = [rogue.protocols.batcher.SplitterV1() for lane in range(numOfAsics)]

for i in range(numOfAsics):
    readerReceiver[i] << readUnbatcher[i] << filter[i]<< fread
    readerReceiver[i].enableDataDebug(True)
    readerReceiver[i].enableDebugPrint(True)
        
def readFromFile(filename) :

    for i in range(4):
        readerReceiver[i].cleanData()
    fread.open(filename)
    fread.closeWait()
    print()
    for i in range(numOfAsics):
        print("ASIC {}: Extracted {} frames".format(i,np.shape(readerReceiver[i].getData())[2]))    

In [ ]:
# If you want to use these helper function from other environments
# you need to run the following command from the command line
# export EPICS_CA_ADDR_LIST=134.79.228.141:5064

# imports and helper functions
from epics import caget, caput
import time

def getKeithleyCurrent() :
    return caget("keithley2400:measCurrAI")

def keithleyEnable(enable):
    caput("keithley2400:enableBO", enable)
    
def getKeithleyVoltage() :
    return caget("keithley2400:measVoltAI")

def decrementKeithleyVoltage() :
    value = caget("keithley2400:measVoltAI.VAL")
    caput('keithley2400:setVoltAO.VAL', value - 1)

def incrementKeithleyVoltage() :
    value = caget("keithley2400:measVoltAI.VAL")
    caput('keithley2400:setVoltAO.VAL', value + 1)

In [ ]:
# If you want to use these helper function from other environments
# you need to run the following command from the command line
# export EPICS_CA_ADDR_LIST=134.79.228.141:5064

# imports and helper functions
from epics import caget, caput

def transferFunction(dataDebugArray, enabledAsics, stepSize=100e-9, 
                     ledDelay = 6e-6, ledRE2AcqRE=2e-6, acqFE2LedRE=2e-6, ledPulseWidth=1.5e-6, ledPulseAmplitude = 2) :

    #cleanup and disable all receivers
    root.enableDataRcv(False)
    root.disableAndCleanAllFullRateDataRcv()
    root.enableDataDebug(False)    

    TRIG.RunTriggerEnable.set(True)
    TRIG.DaqTriggerEnable.set(True)

    daqRE20DelayAcqRE=1.0628e-06 # constant

    acquireWidth = REGCTRL.AcqWidth1_us.get() * 1e-6
    acquireDelay = REGCTRL.AcqDelay1_us.get() * 1e-6
    generalDelay = 0
    
    #select AsicAcq
    REGCTRL.DebugSel0.set(3)

    AcqREStart = daqRE20DelayAcqRE + acquireDelay
    ledREStart = AcqREStart - ledRE2AcqRE
    if (ledREStart < 0 ) :
        generalDelay = ledREStart
        ledREStart = 0
        print("ledRE2AcqRE larger than maximum permitted delay {}. Forcing to 0".format(AcqREStart))
        
    ledREFinish = acquireWidth + AcqREStart + acqFE2LedRE

    print(f'AcqREStart={AcqREStart}, AcqWidth={acquireWidth}, ledREStart={ledREStart}, ledREFinish={ledREFinish}')
    #trigger single shot
    caput("IC-LAB:TrigGen:TriggerSrc", "Single Shot")
    # set internal trigger rate. Make rate slower than time required to acquire images
    caput("IC-LAB:TrigGen:BurstMode", "Disabled")

        #enable asics
    root.enableAllAsics(False)
    for asic in enabledAsics:
        root.enableAsic(asic, True)
        
    #prepare datadebug array
    for i, dataDebug in enumerate(dataDebugArray):
        dataDebug.enableDataDebug(True)
        dataDebug.cleanData()
      
    caput("IC-LAB:TrigGen:A:Relation", "T0")
    caput("IC-LAB:TrigGen:B:Relation", "A") 
    caput("IC-LAB:TrigGen:C:Relation", "T0")
    caput("IC-LAB:TrigGen:D:Relation", "C")
    caput("IC-LAB:TrigGen:E:Relation", "T0")
    caput("IC-LAB:TrigGen:F:Relation", "E")

    time.sleep(1.1)
    
    # AB Daq Trigger
    caput("IC-LAB:TrigGen:AB:Amplitude", 3)
    caput("IC-LAB:TrigGen:CD:Amplitude", 3)
    caput("IC-LAB:TrigGen:EF:Amplitude", ledPulseAmplitude)
    
    caput("IC-LAB:TrigGen:AB:Offset",0)
    caput("IC-LAB:TrigGen:CD:Offset",0)
    caput("IC-LAB:TrigGen:EF:Offset",0)
    
    caput("IC-LAB:TrigGen:A:Delay", ledDelay - generalDelay)
    caput("IC-LAB:TrigGen:B:Delay", 1e-6)
    caput("IC-LAB:TrigGen:C:Delay", ledDelay - generalDelay)
    caput("IC-LAB:TrigGen:D:Delay", 1e-6)
    caput("IC-LAB:TrigGen:E:Delay", 0)
    caput("IC-LAB:TrigGen:F:Delay", ledPulseWidth)
    triggerCount = 0

    steps = (ledREFinish - ledREStart)/stepSize 
    delays = [0] * int(steps+10)
    
    while ledREStart < ledREFinish :
        # EF is led
        caput("IC-LAB:TrigGen:E:Delay", ledREStart)
        delays[triggerCount] = ledREStart
        
        time.sleep(0.02)
        #Enable trigger. Set to internal
        caput("IC-LAB:TrigGen:SoftTrigger", 1)

        ledREStart = ledREStart + stepSize
        triggerCount = triggerCount + 1
        print("{} Trigger sent".format(triggerCount), end='\r')
        #input()
        time.sleep(0.1)
    print("\n")
    for index, dataDebug in enumerate(dataDebugArray):
        print("Received {} on {}".format(dataDebug.getData().shape[2], dataDebug.name), end='\n')

    return delays

    

In [1]:
def enableBurst(enable) :
    caput("IC-LAB:TrigGen:BurstMode", enable)

def setBurstPeriod(period) : 
    caput("IC-LAB:TrigGen:BurstPeriod", period)

def setBurstCount(count) : 
    caput("IC-LAB:TrigGen:BurstCount", count)